In [ ]:
import plotly.graph_objects as go

from src.projects.Chemotaxis.classes.Bacteria import EscherichiaColy, Bacteria
from src.projects.Chemotaxis.classes.Body import Body
from src.projects.Chemotaxis.classes.Cell import Glucose, Lactose, AminoAcid, OxygenBubble

In [ ]:
body = Body(cell_specs=[
    (EscherichiaColy, 2), (Glucose, 4), (Lactose, 2), (AminoAcid, 5), (OxygenBubble, 4)
])

In [ ]:
path = body.chemotaxis(1000)

In [ ]:
fig = go.Figure()

for cell, positions in path.items():
    xs, ys = zip(*positions)
    name = type(cell).__name__

    if isinstance(cell, Bacteria):
        fig.add_trace(go.Scatter(
            x=xs, y=ys, mode='lines',
            name=f"Bacteria ({name})",
            line=dict(width=3)
        ))
        # Marker Start
        fig.add_trace(go.Scatter(
            x=[xs[0]], y=[ys[0]],
            mode='markers', name="Start",
            marker=dict(color='green', size=10, symbol='circle'),
            showlegend=False
        ))
        # Marker End
        fig.add_trace(go.Scatter(
            x=[xs[-1]], y=[ys[-1]],
            mode='markers', name="End",
            marker=dict(color='red', size=10, symbol='x'),
            showlegend=False
        ))
    else:
        fig.add_trace(go.Scatter(
            x=xs, y=ys, mode='lines',
            name=f"Target ({name})",
            line=dict(dash='dash')
        ))
        # Marker Start pour les targets
        fig.add_trace(go.Scatter(
            x=[xs[0]], y=[ys[0]],
            mode='markers', name="Start",
            marker=dict(color='blue', size=8, symbol='square'),
            showlegend=False
        ))
        # Marker End pour les targets
        fig.add_trace(go.Scatter(
            x=[xs[-1]], y=[ys[-1]],
            mode='markers', name="End",
            marker=dict(color='orange', size=8, symbol='x'),
            showlegend=False
        ))

fig.update_layout(
    title="Chemotaxis Path of Cells",
    xaxis=dict(title="X Position", range=[-body.size / 2, body.size / 2]),
    yaxis=dict(title="Y Position", range=[-body.size / 2, body.size / 2], scaleanchor="x", scaleratio=1),
    showlegend=True,
)

fig.update_yaxes(scaleanchor="x", scaleratio=1)
fig.show()

In [ ]:
def plot_chemotaxis(path, body, plot_trajectory=True, plot_step=10):
    cells = list(path.keys())
    steps = len(next(iter(path.values())))
    fig = go.Figure()

    cell_colors = {
        'EscherichiaColy': 'red',
        'Glucose': 'blue',
        'Lactose': 'white',
        'AminoAcid': 'purple',
        'OxygenBubble': 'green'
    }

    initial_data = []
    for cell in cells:
        name = type(cell).__name__
        xs, ys = zip(*path[cell])
        symbol = 'circle-x' if isinstance(cell, Bacteria) else 'hexagram'

        # Trajectoire initiale (T=0)
        if plot_trajectory:
            initial_data.append(go.Scatter(
                x=[xs[0]], y=[ys[0]],
                mode='lines',
                line=dict(color=cell_colors[name], width=2),
                showlegend=False
            ))

        # Position initiale
        initial_data.append(go.Scatter(
            x=[xs[0]], y=[ys[0]],
            mode='markers',
            marker=dict(size=12, color=cell_colors[name], symbol=symbol, line=dict(width=1, color='black')),
            name=f"{name}",
            showlegend=True
        ))

    fig.add_traces(initial_data)

    # --- Animation frames ---
    frames = []
    for t in range(0, steps, plot_step):
        frame_data = []
        for cell in cells:
            name = type(cell).__name__
            xs, ys = zip(*path[cell])
            symbol = 'circle-x' if isinstance(cell, Bacteria) else 'hexagram'

            if plot_trajectory:
                frame_data.append(go.Scatter(
                    x=xs[:t + 1], y=ys[:t + 1],
                    mode='lines',
                    line=dict(color=cell_colors[name], width=2),
                    showlegend=False
                ))

            frame_data.append(go.Scatter(
                x=[xs[t]], y=[ys[t]],
                mode='markers',
                marker=dict(size=12, color=cell_colors[name], symbol=symbol, line=dict(width=1, color='black')),
                name=f"{name}",
                showlegend=True
            ))

        frames.append(go.Frame(data=frame_data, name=str(t)))

    fig.frames = frames

    # --- Layout ---
    fig.update_layout(
        title="Chemotaxis – Trajectoires évolutives",
        xaxis=dict(title="X", range=[-body.size / 2, body.size / 2], scaleanchor="y", scaleratio=1),
        yaxis=dict(title="Y", range=[-body.size / 2, body.size / 2]),
        updatemenus=[{
            "type": "buttons",
            "buttons": [
                {
                    "label": "Play",
                    "method": "animate",
                    "args": [None, {"frame": {"duration": 100, "redraw": True}, "fromcurrent": True}]
                },
                {
                    "label": "Pause",
                    "method": "animate",
                    "args": [[None], {"mode": "immediate", "frame": {"duration": 0, "redraw": False}}]
                }
            ]
        }],
        sliders=[{
            "steps": [
                {
                    "method": "animate",
                    "args": [[str(t)], {"mode": "immediate", "frame": {"duration": 0, "redraw": True}}],
                    "label": str(t)
                } for t in range(0, steps, plot_step)
            ],
            "transition": {"duration": 0},
            "x": 0, "y": -0.1,
            "currentvalue": {"prefix": "T = "}
        }]
    )

    fig.show()

In [ ]:
plot_chemotaxis(path, body, plot_trajectory=True, plot_step=10)